In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

In [ ]:
# 1. Download and Explore Dataset
# Neighborhood has a total of 5 boroughs and 306 neighborhoods. 
# In order to segement the neighborhoods and explore them, we will essentially need a dataset 
# that contains the 5 boroughs and the neighborhoods that exist in each borough as well as 
# the latitude and logitude coordinates of each neighborhood.

# Luckily, this dataset exists for free on the web. 
# Feel free to try to find this dataset on your own, but here is the link to the dataset: 
#     https://geo.nyu.edu/catalog/nyu_2451_34572

In [ ]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

In [ ]:
# Load and explore the data
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [ ]:
newyork_data

In [ ]:
neighborhoods_data = newyork_data['features']

In [ ]:
neighborhoods_data[0]

In [ ]:
# define the dataframe columns
columns_names = ['Borough','Neighboorhood','Latitude','Longitude']

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=columns_names)

In [ ]:
neighborhoods

In [ ]:
# Then let's loop through the data and fill the dataframe one row at a time.

for data in neighborhoods_data:
    borough = data['properties']['borough']
    neighborhood_name = data['properties']['name']
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Longitude': neighborhood_lon,
                                          'Latitude': neighborhood_lat},
                                           ignore_index=True)

In [ ]:
# Quickly examine the resulting dataframe.
neighborhoods.rename(index=str, columns={'Neighboorhood': 'Neighborhood'},inplace=True)
neighborhoods.head(5)

In [ ]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

In [ ]:
# Use geopy library to get the latitude and longitude values of New York City

address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explore")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geograpical coordinate of New York City are {}, {}.'.format(latitude,longitude))

In [ ]:
# Create a map of New York using latitude and longitude values

map_newyork= folium.Map(location=[latitude,longitude],zoom_start=10)

# add markers to map

for lat,lng,borough,neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{},{}'.format(neighborhood,borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)

# show the map
map_newyork

In [ ]:
# slice the nighborhoods in Manhattan

manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

In [ ]:
# Let's get the geographical coordinates of Manhattan.

address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

In [ ]:
# As we did with all of New York City, let's visualizat Manhattan the neighborhoods in it.

# create map of Manhattan using latitude and longitude values

map_manhattan = folium.Map(location = [latitude,longitude], zoom_start = 11)

# add markers to map

for lat,lng,label in zip(manhattan_data['Latitude'],manhattan_data['Longitude'],manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)
    
#show map
map_manhattan

In [ ]:
# Define Foursquare Credentials and Version

CLIENT_ID = 'THW4T43GANDRLSNMXNCEK1ZGIBT0T2ONW3UXXXPVFBR2LS2Y' # your Foursquare ID
CLIENT_SECRET = 'X3IFLAKKQQCOKA40VJVJS31FPHHAGP5O150C0WUENFZ1UPNF' # your Foursquare Secret
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
# Let's explore the first neighborhood in our dataframe

manhattan_data.loc[0,'Neighborhood']

In [ ]:
neighborhood_latitude = manhattan_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = manhattan_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = manhattan_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

In [ ]:
#Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters

LIMIT = 100

radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

url

In [ ]:
# send GET request and examine the result

results = requests.get(url).json()

results

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) # flattern JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:,filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis = 1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

In [ ]:
# 2. Explore Neighborhoods in Manhattan
# Let's create a function to repeat the same process to all the neighborhoods in Manhattan

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only  relevant information for each nearby venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return (nearby_venues)

In [ ]:
# Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues.
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

In [ ]:
# Let's check the size of the resulting dataframe

print(manhattan_venues.shape)
manhattan_venues.head()

In [ ]:
# Let's check how many venues were returned for each neighborhood
manhattan_venues.groupby('Neighborhood').count()

In [ ]:
# Let's find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

In [ ]:
# 3. Analyze Each Neighborhood

In [ ]:
# one hot encoding 
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="",prefix_sep="")

# add nieghborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

In [ ]:
manhattan_onehot.shape

In [ ]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

In [ ]:
manhattan_grouped.shape

In [ ]:
# Let's print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----" + hood + "----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
# Let's put that into a pandas dataframe

In [ ]:
# sor the venues in descending order

def return_most_common_venues(row,num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st','nd','rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind,1:] = return_most_common_venues(manhattan_grouped.iloc[ind,:], num_top_venues)
    
neighborhoods_venues_sorted.head()

In [ ]:
# 4. Cluster Neighborhoods

In [ ]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood',1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
# Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged['Cluster Labels'].unique()

In [ ]:
manhattan_merged.head()

In [ ]:
# Let's visualize the resulting clusters

# create a map

map_clusters = folium.Map(location=[latitude,longitude],zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
color_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in color_array]

# add markers to the map
markers_colors = []

for lat, lng, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + 'Cluster ' + str(cluster), parse_html = True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

In [ ]:
# 5. Examine Clusters

In [ ]:
# Cluster 1
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0 , manhattan_merged.columns[[1] +list(range(5, manhattan_merged.shape[1]))]]
                                                                                        
                                                                                        

In [ ]:
# Cluster 2
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1 , manhattan_merged.columns[[1] +list(range(5, manhattan_merged.shape[1]))]]
                                                                                        
                  

In [ ]:
# Cluster 3
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2 , manhattan_merged.columns[[1] +list(range(5, manhattan_merged.shape[1]))]]
                                                                                        
                  

In [ ]:
# Cluster 4
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3 , manhattan_merged.columns[[1] +list(range(5, manhattan_merged.shape[1]))]]
                                                                                        
                  

In [ ]:
# Cluster 5
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4 , manhattan_merged.columns[[1] +list(range(5, manhattan_merged.shape[1]))]]
                                                                                        
                  